# Credit Risk Score


- Binary classification

$$g(x_{i}) \approx y_{i}$$

$$y_{i} \in \{0, 1\}$$

- $1$: Default
- $0$: No default

**Dataset**:

[kaggle-credit-scoring](https://www.kaggle.com/datasets/nightcrawler101/creditscoring-csv?select=CreditScoring.csv) or [github-credit-scoring](https://github.com/gastonstat/CreditScoring)


Install packages


In [5]:
!uv pip install -q \
    python-dotenv==1.2.1 \
    pandas==2.3.2 \
    pandas-stubs==2.3.2.250827 \
    numpy==2.3.2 \
    matplotlib==3.10.6 \
    seaborn==0.13.2 \
    scikit-learn==1.7.1 \
    tqdm==4.67.1

Append notebooks directory to sys.path


In [ ]:
import sys

sys.path.append("../../..")

In [7]:
import os
import pathlib
import random
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
from typing import Tuple, Union
import numpy as np
from numpy.typing import NDArray
import seaborn as sns
import datetime
import pickle
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from notebooks.python.utils.data_extraction.data_extraction import (
    KaggleDataExtractor,
    KaggleExtractionConfig,
)

pd.set_option("display.max_columns", None)

sns.set_style("darkgrid")
sns.set_theme(style="darkgrid")

%matplotlib inline

load_dotenv()  # Root directory .env file

True

## Utility scripts:

**KaggleDataExtractor**:

```py
--8<-- "docs/notebooks/python/utils/data_extraction/data_extraction.py"
```


Create data directory


In [ ]:
DATA_DIR = pathlib.Path("data/credit-risk-score")

os.makedirs(DATA_DIR, exist_ok=True)

Download dataset from Kaggle


In [ ]:
username = os.getenv("KAGGLE_USERNAME")
api_token = os.getenv("KAGGLE_API_TOKEN")
file_name = "CreditScoring.csv"

extractor = KaggleDataExtractor(username=username, api_token=api_token)

config = KaggleExtractionConfig(
    dataset_slug="nightcrawler101/creditscoring-csv",
    file_name=file_name,
    destination_path=DATA_DIR,
    output_file_name="credit-scoring.csv",
)

if not os.path.isfile(DATA_DIR / "credit-scoring.csv"):
    extractor.download_dataset(config)

## Data Preparation


Load dataset


In [ ]:
df = pd.read_csv(DATA_DIR / "credit-scoring.csv")

df.head(n=2)

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658


Inspect all columns at once


In [ ]:
df.head(3).T

,0,1,2
Status,1,1,2
Seniority,9,17,10
Home,1,1,2
Time,60,60,36
Age,30,58,46
Marital,2,3,2
Records,1,1,2
Job,3,1,3
Expenses,73,48,90
Income,129,131,200


Data summary


In [ ]:
df_summary = pd.DataFrame(
    {
        "column": df.columns,
        "dtype": [df[col].dtype for col in df.columns],
        "sample_unique": [df[col].unique()[:6] for col in df.columns],
        "n_unique": [df[col].nunique() for col in df.columns],
    }
)
df_summary

,column,dtype,sample_unique,n_unique
0,Status,int64,"[1, 2, 0]",3
1,Seniority,int64,"[9, 17, 10, 0, 1, 29]",47
2,Home,int64,"[1, 2, 5, 3, 6, 4]",7
3,Time,int64,"[60, 36, 12, 48, 18, 24]",11
4,Age,int64,"[30, 58, 46, 24, 26, 36]",50
5,Marital,int64,"[2, 3, 1, 4, 5, 0]",6
6,Records,int64,"[1, 2]",2
7,Job,int64,"[3, 1, 2, 0, 4]",5
8,Expenses,int64,"[73, 48, 90, 63, 46, 75]",94
9,Income,int64,"[129, 131, 200, 182, 107, 214]",353


Clean column names


In [ ]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

df.head(n=2)

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
